In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [2]:
df = pd.read_csv('final_project_dataset.csv')

In [3]:
df.describe

<bound method NDFrame.describe of         encounter_id  patient_nbr             race  gender      age weight  \
0            2278392      8222157        Caucasian  Female   [0-10)      ?   
1             149190     55629189        Caucasian  Female  [10-20)      ?   
2              64410     86047875  AfricanAmerican  Female  [20-30)      ?   
3             500364     82442376        Caucasian    Male  [30-40)      ?   
4              16680     42519267        Caucasian    Male  [40-50)      ?   
...              ...          ...              ...     ...      ...    ...   
143419     443854148     41088789        Caucasian    Male  [70-80)      ?   
143420     443857166     31693671        Caucasian  Female  [80-90)      ?   
143421     443857166     31693671        Caucasian  Female  [80-90)      ?   
143422     443857166     31693671        Caucasian  Female  [80-90)      ?   
143423     443867222    175429310        Caucasian    Male  [70-80)      ?   

        admission_type_id  di

In [6]:
# df['weight'].fillna(df['weight'].mode()[0], inplace=True)
# df['age'].fillna(df['age'].mode()[0], inplace=True)

C:\Users\steve\AppData\Local\Temp\ipykernel_27352\2023658374.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['weight'].fillna(df['weight'].mode()[0], inplace=True)


In [4]:
men = sum(df['gender'] == 'Male')
women = sum(df['gender'] == 'Female')
print(men, women)

67234 76185


Fill missing numerical values with the mean

In [5]:
categorical_features = ["gender", "race"]
numerical_columns = ['age', 'weight', 'num_lab_procedures', 'number_diagnoses', 'num_medications', 'num_procedures']

for col in categorical_features:
    df.fillna({col: df[col].mode()[0]}, inplace=True)

for col in numerical_columns:
    if col not in['weight', 'age']:
        df.fillna({col: df[col].mean()}, inplace=True)

In [6]:
# Define target variable (readmission: 1 if "<30", else 0)
df["readmitted"] = df["readmitted"].apply(lambda x: 1 if x == "<30" else 0)

# Select relevant features (excluding identifiers and redundant columns)
features = [
    "age", "gender", "race", "admission_type_id", "discharge_disposition_id", 
    "admission_source_id", "time_in_hospital", "num_lab_procedures", "num_procedures", 
    "num_medications", "number_diagnoses"
]

 

# Handle missing values (replace "?" with NaN and fill with mode for categorical, mean for numerical)
df.replace("?", np.nan, inplace=True)
for col in features:
    if df[col].dtype == "object":
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:
        df[col].fillna(df[col].mean(), inplace=True)

# Convert categorical variables using OneHotEncoding
categorical_features = ["gender", "race"]
 


C:\Users\steve\AppData\Local\Temp\ipykernel_14096\3182967113.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
C:\Users\steve\AppData\Local\Temp\ipykernel_14096\3182967113.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

In [8]:
df = df.drop([
    'encounter_id', 
    'patient_nbr',
    'ndc_code'
], axis=1)

In [9]:
df = df.replace('?', pd.NA)

In [10]:
df.columns

Index(['race', 'gender', 'age', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'primary_diagnosis_code',
       'other_diagnosis_codes', 'number_outpatient', 'number_inpatient',
       'number_emergency', 'num_lab_procedures', 'number_diagnoses',
       'num_medications', 'num_procedures', 'max_glu_serum', 'A1Cresult',
       'change', 'readmitted'],
      dtype='object')

In [7]:
# Split features and target
X = df.drop('readmitted', axis=1)
y = df['readmitted']

In [8]:
X.columns

Index(['encounter_id', 'patient_nbr', 'race', 'gender', 'age', 'weight',
       'admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'payer_code', 'medical_specialty',
       'primary_diagnosis_code', 'other_diagnosis_codes', 'number_outpatient',
       'number_inpatient', 'number_emergency', 'num_lab_procedures',
       'number_diagnoses', 'num_medications', 'num_procedures', 'ndc_code',
       'max_glu_serum', 'A1Cresult', 'change'],
      dtype='object')

Logistic Regression

In [10]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Define numeric and categorical features
numeric_features = [
    'time_in_hospital', 
    'num_lab_procedures',
    'num_medications',
    'number_emergency',
    'number_diagnoses'
]

categorical_features = [
    'race', 
    'gender',
    'age',
    'admission_type_id',
    'discharge_disposition_id',
    'admission_source_id',
    'max_glu_serum',
    'A1Cresult',
    'change'
]

# Preprocessing pipeline for numeric and categorical features
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_features)  # Handling unknown values
])

# Full pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle missing values
    ('classifier', LogisticRegression(
        class_weight='balanced',
        random_state=42
    ))
])



# Train/Test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42,
    stratify=y
)

# Train model
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

# Output evaluation metrics
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("\nROC AUC Score:", roc_auc_score(y_test, y_proba))


Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.60      0.72     38305
           1       0.14      0.53      0.22      4723

    accuracy                           0.59     43028
   macro avg       0.53      0.56      0.47     43028
weighted avg       0.83      0.59      0.67     43028


ROC AUC Score: 0.5903514098910195


In [11]:
roc_auc_score(y_test, y_proba, multi_class='ovr')


0.5903514098910195

Random Forest

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline


numeric_features = [
    'time_in_hospital', 
    'num_lab_procedures',
    'num_medications',
    'number_emergency',
    'number_diagnoses'
]

categorical_features = [
    'race', 
    'gender',
    'age',
    'admission_type_id',
    'discharge_disposition_id',
    'admission_source_id',
    'max_glu_serum',
    'A1Cresult',
    'change'
]

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
])

# Full pipeline
model = Pipeline([
    ('preprocessor', preprocessor),
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Handle remaining NaNs
    ('classifier', RandomForestClassifier(
        n_estimators=100,
        max_depth=10,  # Adjust this based on performance
        class_weight='balanced',
        random_state=42
    ))
])

# Train/Test split
# =================================================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3, 
    random_state=42,
    stratify=y
)

# Train model
# =================================================================
model.fit(X_train, y_train)

# Evaluate
# =================================================================
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print("Classification Report:")
print(classification_report(y_test, y_pred))
print("\nROC AUC Score:", roc_auc_score(y_test, y_proba))

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.65      0.76     38305
           1       0.17      0.56      0.26      4723

    accuracy                           0.64     43028
   macro avg       0.54      0.61      0.51     43028
weighted avg       0.84      0.64      0.71     43028


ROC AUC Score: 0.6528003350090512


For Random Forest Classifiers

In [13]:
# Get feature names after preprocessing
feature_names = model.named_steps['preprocessor'].get_feature_names_out()

# Get feature importances from the Random Forest model
importances = model.named_steps['classifier'].feature_importances_

# Create a DataFrame for feature importances
feature_importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Filter for gender-related features
gender_importance = feature_importance_df[feature_importance_df['Feature'].str.contains('gender')]
race_importance = feature_importance_df[feature_importance_df['Feature'].str.contains('race')]

print(gender_importance)
print(race_importance)


                        Feature  Importance
10           cat__gender_Female    0.009547
11             cat__gender_Male    0.009749
12  cat__gender_Unknown/Invalid    0.000011
                     Feature  Importance
5  cat__race_AfricanAmerican    0.008721
6            cat__race_Asian    0.003007
7        cat__race_Caucasian    0.008768
8         cat__race_Hispanic    0.005981
9            cat__race_Other    0.003593


For logistic regression models

In [15]:
# Get feature names after preprocessing
feature_names = model.named_steps['preprocessor'].get_feature_names_out()

# Get coefficients from the logistic regression model
coefficients = model.named_steps['classifier'].coef_[0]

# Create a DataFrame for coefficients
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Filter for gender-related coefficients
gender_coefs = coef_df[coef_df['Feature'].str.contains('gender')]
print(gender_coefs)

AttributeError: 'RandomForestClassifier' object has no attribute 'coef_'

In [ ]:
race_coefs = coef_df[coef_df['Feature'].str.contains('race')]
print(race_coefs)

Check coefficients and equalized odds, demographic parity for gender

In [16]:
from fairlearn.metrics import equalized_odds_difference, demographic_parity_difference, demographic_parity_ratio

# Predictions
y_pred = model.predict(X_test)

# Equalized Odds Difference
eod = equalized_odds_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test['gender']  # Use the original gender column
)

# Demographic Parity Difference
dpd = demographic_parity_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test['gender']
)
# Disparate Impact Ratio (now called Demographic Parity Ratio)
di_ratio = demographic_parity_ratio(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test['gender']
)

print(f"Demographic Parity Ratio: {di_ratio:.4f}")
print(f"Equalized Odds Difference: {eod:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")

Demographic Parity Ratio: 0.8090
Equalized Odds Difference: 0.6009
Demographic Parity Difference: 0.0787


Check coefficients and equalized odds, demographic parity for race

In [17]:
eod = equalized_odds_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test['race']  # Use the original gender column
)

# Demographic Parity Difference
dpd = demographic_parity_difference(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test['race']
)

print(f"Equalized Odds Difference: {eod:.4f}")
print(f"Demographic Parity Difference: {dpd:.4f}")



di_ratio = demographic_parity_ratio(
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=X_test['race']
)

print(f"Demographic Parity Ratio: {di_ratio:.4f}")

Equalized Odds Difference: 0.2546
Demographic Parity Difference: 0.2489
Demographic Parity Ratio: 0.3629


In [18]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA

# List of categorical columns
categorical_cols = ['race', 'gender', 'payer_code', 'medical_specialty', 
                    'primary_diagnosis_code', 'other_diagnosis_codes', 
                    'max_glu_serum', 'A1Cresult', 'change']

# One-hot encode categorical variables


encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = encoder.fit_transform(X_train[categorical_cols])

# Convert the encoded array into a DataFrame
X_encoded_df = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out())

# Drop original categorical columns and concatenate encoded features
X_train = X_train.drop(columns=categorical_cols).reset_index(drop=True)
X_train = pd.concat([X_train, X_encoded_df], axis=1)


pca = PCA(n_components=500)  # Keep only 500 features
X_train_reduced = pca.fit_transform(X_train)


# Now apply SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)


MemoryError: Unable to allocate 13.0 GiB for an array with shape (17353, 100396) and data type object

In [43]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, classification_report

smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# Train a Random Forest model
model = RandomForestClassifier(class_weight='balanced', random_state=42)

# Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='roc_auc')
grid_search.fit(X_resampled, y_resampled)

# Evaluate
y_pred = grid_search.predict(X_test)
y_proba = grid_search.predict_proba(X_test)[:, 1]
print("ROC AUC Score:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred))

ValueError: could not convert string to float: 'Caucasian'